<span style="color: purple">

Load in stored variables:

</span>

In [2]:
%store -r data_dir sites_gdf siskiyou_forest_gdf padres_forest_gdf

<span style="color: purple">

Import packages:

</span>

In [3]:
# Import necessary packages
import os

import matplotlib.pyplot as plt # Overlay pandas and xarray plots
import pandas as pd # Aggregating and data manipulation
from tqdm.notebook import tqdm # Progress bars on loops
import xarray as xr

## STEP 2c: DATA ACCESS - CLIMATE MODELS

You can use MACAv2 data for historical and future climate data. Be sure
to compare at least two 30-year time periods (e.g. historical vs. 10
years in the future) for at least four of the CMIP models. Overall, you
should be downloading at least 8 climate rasters for each of your sites,
for a total of 16. **You will *need* to use loops and/or functions to do
this cleanly!**.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Write a <strong>function with a numpy-style docstring</strong> that
will download MACAv2 data for a particular climate model, emissions
scenario, spatial domain, and time frame. Then, use loops to download
and organize the 16+ rasters you will need to complete this section. The
<a
href="http://thredds.northwestknowledge.net:8080/thredds/reacch_climate_CMIP5_macav2_catalog2.html">MACAv2
dataset is accessible from their Thredds server</a>. Include an
arrangement of sites, models, emissions scenarios, and time periods that
will help you to answer your scientific question.</p></div></div>

<span style="color: purple">

List of climate models:

</span>

<span style="color: magenta">

* Warm and wet: CanESM2

</span>

<span style="color: limegreen">

* Warm and dry: MIROC-ESM-CHEM

</span>

<span style="color: teal">

* Cold and wet: MRI-CGCM3

</span>

<span style="color: black">

* Cold and dry: GFDL-ESM2M

</span>

<span style="color: purple">

<span style="color: purple">

The time periods I will compare are:

* 2036 to 2065 - this will allow for a summary of climate for 2050
* 2066 to 2095 - this will allow for a summary of climate for 2080

Climate variable I will focus on: the average annual precipitation.
</span>

In [4]:
# Make MACA data directory
maca_dir = os.path.join(data_dir, 'maca-dir')
os.makedirs(maca_dir, exist_ok=True)
maca_dir

maca_pattern = os.path.join(maca_dir, '*.nc')
maca_pattern

'C:\\Users\\riede\\earth-analytics\\data\\spring-2025-habitat-suitability\\maca-dir\\*.nc'

<span style="color: purple">

Define functions:

</span>

In [5]:
# Define function to convert longitude/latitude
def convert_longitude(longitude):
    """
    Convert longitude range from 0-360 to -180-180
    
    Parameters
    ----------
    longitude : float
        `longitude` values of dataset
    
    Returns
    -------
    longitude : float
        Longitude values converted from 0-360 to -180-180"""
    return (longitude - 360) if longitude > 180 else longitude

In [ ]:
def process_maca_da(site_dict,
                    years_list,
                    models_list,
                    rcp_value,
                    maca_data_dir):
    """
    Create a URL to access MACA climate precipitation data from the Northwest Knowledge Network.
    Use the URL to download, open, squeeze, and crop raster climate precipitation data.
    
    Parameters
    ----------
    site_dict : dict
        Dictionary of site GeoDataFrames
    years_list : list
        List of 5 year time periods
    models_list : list
        List of climate model names
    rcp_value : str
        Representative Concentration Pathway value
    maca_data_dir: str
        file path where maca NetCDF files will be stored

    Returns
    -------
    maca_da_list : list
        A list of DataArrays of climate data (precipitation) for
        the different rasters covering the area of interest
    """
    # empty list to store processed MACA DataArrays
    maca_da_list = []

    for site_name, site_gdf in tqdm(site_dict.items()):
        for date_range in tqdm(years_list):
            for model in tqdm(models_list):
                # Define MACA url
                maca_url = (
                    'http://thredds.northwestknowledge.net:8080/thredds/dodsC'
                    '/MACAV2'
                    f'/{model}'
                    '/macav2metdata_pr'
                    f'_{model}_r1i1p1'
                    f'_{rcp_value}'
                    f'_{date_range}_CONUS'
                    '_monthly.nc')
                # check maca_url
                print(maca_url)

                # Only download MACA data once
                maca_path = os.path.join(maca_data_dir, f'maca_{model}_{site_name}_{rcp_value}_{date_range}_CONUS_monthly.nc')
                print(maca_path)
                if not os.path.exists(maca_path):
                    maca_da = xr.open_dataset(maca_url).squeeze().precipitation
                    maca_da.to_netcdf(maca_path)

                # Open & Squeeze MACA dataset
                maca_da = xr.open_dataset(maca_url).squeeze().precipitation
                print('Opened & Squeezed')

                # Define bounds
                bounds_maca = (site_gdf
                                .to_crs(maca_da.rio.crs)
                                .total_bounds)

                # Change maca_ds longitude values to match the foest gdfs
                maca_da = maca_da.assign_coords(
                    lon=("lon", [convert_longitude(l) for l in maca_da.lon.values]))

                # Set spatial dimensions of maca_da
                maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')

                # Crop maca_da
                maca_da = maca_da.rio.clip_box(*bounds_maca)
                print('Cropped')

                # Add cropped da to a dictionary w/ metadata and then the maca_da_list
                maca_da_list.append(dict(
                    site_name=site_name,
                    climate_model=model,
                    date_range=date_range,
                    da=maca_da))
    return maca_da_list

<span style="color: purple">

Set up variables to use in process_maca_da function:

</span>

In [7]:
# site dictionary
siskiyou_padres_dict = {
    'SiskiyouForest': siskiyou_forest_gdf,
    'LosPadresForest': padres_forest_gdf
}

# list of time periods to create MACA urls
years_2050 = ['2036_2040', '2041_2045', '2046_2050',
         '2051_2055', '2056_2060', '2061_2065']

years_2080 = ['2066_2070', '2071_2075', '2076_2080',
              '2081_2085', '2086_2090', '2091_2095']

# list of climate models
climate_models = ['CanESM2', 'MIROC-ESM-CHEM',
          'MRI-CGCM3', 'GFDL-ESM2M']

<span style="color: purple">

Process MACA DataArrays for 2050 time period for both sites for all 4 climate models:

</span>

In [8]:
maca_pr_2050_da_list = process_maca_da(siskiyou_padres_dict,
                                       years_2050,
                                       climate_models,
                                       'rcp85',
                                       maca_dir)

maca_pr_2050_da_list

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_SiskiyouForest_rcp85_2036_2040_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_SiskiyouForest_rcp85_2036_2040_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_SiskiyouForest_rcp85_2036_2040_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFDL-E

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2056_2060_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_LosPadresForest_rcp85_2056_2060_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2056_2060_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_LosPadresForest_rcp85_2056_2060_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2056_2060_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_LosPadresForest_rcp85_2056_2060_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFD

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2061_2065_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_LosPadresForest_rcp85_2061_2065_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2061_2065_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_LosPadresForest_rcp85_2061_2065_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2061_2065_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_LosPadresForest_rcp85_2061_2065_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFD

[{'site_name': 'SiskiyouForest',
  'climate_model': 'CanESM2',
  'date_range': '2036_2040',
  'da': <xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
  [40500 values with dtype=float32]
  Coordinates:
    * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
    * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
    * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
      crs      int64 8B 0
  Attributes:
      long_name:      Monthly Precipitation Amount
      units:          mm
      standard_name:  precipitation
      cell_methods:   time: sum(interval: 24 hours): sum over days
      comments:       Total monthly precipitation at surface: includes both liq...
      _ChunkSizes:    [ 10  44 107]},
 {'site_name': 'SiskiyouForest',
  'climate_model': 'MIROC-ESM-CHEM',
  'date_range': '2036_2040',
  'da': <xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
  [40500 value

In [9]:
maca_pr_2050_df = pd.DataFrame(maca_pr_2050_da_list)

# Display maca_df w/o DataArray column
display(maca_pr_2050_df.drop('da', axis='columns'))

# Pull out the data array for the first row
maca_pr_2050_df.da.values[0]

,site_name,climate_model,date_range
0,SiskiyouForest,CanESM2,2036_2040
1,SiskiyouForest,MIROC-ESM-CHEM,2036_2040
2,SiskiyouForest,MRI-CGCM3,2036_2040
3,SiskiyouForest,GFDL-ESM2M,2036_2040
4,SiskiyouForest,CanESM2,2041_2045
5,SiskiyouForest,MIROC-ESM-CHEM,2041_2045
6,SiskiyouForest,MRI-CGCM3,2041_2045
7,SiskiyouForest,GFDL-ESM2M,2041_2045
8,SiskiyouForest,CanESM2,2046_2050
9,SiskiyouForest,MIROC-ESM-CHEM,2046_2050


<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<span style="color: purple">

Process MACA DataArrays for 2080 time period for both sites for all 4 climate models:

</span>

In [10]:
maca_pr_2080_da_list = process_maca_da(siskiyou_padres_dict,
                                       years_2080,
                                       climate_models,
                                       'rcp85',
                                       maca_dir)

maca_pr_2080_da_list

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2066_2070_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_SiskiyouForest_rcp85_2066_2070_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2066_2070_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_SiskiyouForest_rcp85_2066_2070_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2066_2070_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_SiskiyouForest_rcp85_2066_2070_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFDL-E

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2086_2090_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_LosPadresForest_rcp85_2086_2090_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2086_2090_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_LosPadresForest_rcp85_2086_2090_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2086_2090_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_LosPadresForest_rcp85_2086_2090_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFD

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2091_2095_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_LosPadresForest_rcp85_2091_2095_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2091_2095_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_LosPadresForest_rcp85_2091_2095_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2091_2095_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_LosPadresForest_rcp85_2091_2095_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFD

[{'site_name': 'SiskiyouForest',
  'climate_model': 'CanESM2',
  'date_range': '2066_2070',
  'da': <xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
  [40500 values with dtype=float32]
  Coordinates:
    * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
    * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
    * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
      crs      int64 8B 0
  Attributes:
      long_name:      Monthly Precipitation Amount
      units:          mm
      standard_name:  precipitation
      cell_methods:   time: sum(interval: 24 hours): sum over days
      comments:       Total monthly precipitation at surface: includes both liq...
      _ChunkSizes:    [ 10  44 107]},
 {'site_name': 'SiskiyouForest',
  'climate_model': 'MIROC-ESM-CHEM',
  'date_range': '2066_2070',
  'da': <xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
  [40500 value

In [11]:
maca_pr_2080_df = pd.DataFrame(maca_pr_2080_da_list)

# Display maca_df w/o DataArray column
display(maca_pr_2080_df.drop('da', axis='columns'))

# Pull out the data array for the first row
maca_pr_2080_df.da.values[0]

,site_name,climate_model,date_range
0,SiskiyouForest,CanESM2,2066_2070
1,SiskiyouForest,MIROC-ESM-CHEM,2066_2070
2,SiskiyouForest,MRI-CGCM3,2066_2070
3,SiskiyouForest,GFDL-ESM2M,2066_2070
4,SiskiyouForest,CanESM2,2071_2075
5,SiskiyouForest,MIROC-ESM-CHEM,2071_2075
6,SiskiyouForest,MRI-CGCM3,2071_2075
7,SiskiyouForest,GFDL-ESM2M,2071_2075
8,SiskiyouForest,CanESM2,2076_2080
9,SiskiyouForest,MIROC-ESM-CHEM,2076_2080


<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<span style="color: purple">

Split each DataFrame for each time period up by model and site. Instead of having two DataFrames for each time period, I will now have 8 for each time period (4 for Siskiyou, 4 for Los Padres). Each DataFrame will represent one of the 4 different climate models, one of the sites, and one of the time periods so that we can analyze the projected amount of precipitation in each time period for each model for each site. 

</span>

In [12]:
# select the Siskiyou rows from the maca_pr_2050_df
siskiyou_2050_df = maca_pr_2050_df[maca_pr_2050_df['site_name']=='SiskiyouForest']

# select the CanESM2 rows from the siskiyou_2050_df
siskiyou_2050_can_df = siskiyou_2050_df[siskiyou_2050_df['climate_model']=='CanESM2']

# select the MIROC-ESM-CHEM rows from the siskiyou_2050_df
siskiyou_2050_miroc_df = siskiyou_2050_df[siskiyou_2050_df['climate_model']=='MIROC-ESM-CHEM']

# select the MRI-CGCM3 rows from the siskiyou_2050_df
siskiyou_2050_mri_df = siskiyou_2050_df[siskiyou_2050_df['climate_model']=='MRI-CGCM3']

# select the GFDL-ESM2M	 rows from the siskiyou_2050_df
siskiyou_2050_gfdl_df = siskiyou_2050_df[siskiyou_2050_df['climate_model']=='GFDL-ESM2M']

# Display df w/o DataArray column
display(siskiyou_2050_can_df.drop('da', axis='columns'))
display(siskiyou_2050_miroc_df.drop('da', axis='columns'))
display(siskiyou_2050_mri_df.drop('da', axis='columns'))
display(siskiyou_2050_gfdl_df.drop('da', axis='columns'))

# Pull out the data array for the first row from the CanESM2 df
siskiyou_2050_can_df.da.values[0]

,site_name,climate_model,date_range
0,SiskiyouForest,CanESM2,2036_2040
4,SiskiyouForest,CanESM2,2041_2045
8,SiskiyouForest,CanESM2,2046_2050
12,SiskiyouForest,CanESM2,2051_2055
16,SiskiyouForest,CanESM2,2056_2060
20,SiskiyouForest,CanESM2,2061_2065


,site_name,climate_model,date_range
1,SiskiyouForest,MIROC-ESM-CHEM,2036_2040
5,SiskiyouForest,MIROC-ESM-CHEM,2041_2045
9,SiskiyouForest,MIROC-ESM-CHEM,2046_2050
13,SiskiyouForest,MIROC-ESM-CHEM,2051_2055
17,SiskiyouForest,MIROC-ESM-CHEM,2056_2060
21,SiskiyouForest,MIROC-ESM-CHEM,2061_2065


,site_name,climate_model,date_range
2,SiskiyouForest,MRI-CGCM3,2036_2040
6,SiskiyouForest,MRI-CGCM3,2041_2045
10,SiskiyouForest,MRI-CGCM3,2046_2050
14,SiskiyouForest,MRI-CGCM3,2051_2055
18,SiskiyouForest,MRI-CGCM3,2056_2060
22,SiskiyouForest,MRI-CGCM3,2061_2065


,site_name,climate_model,date_range
3,SiskiyouForest,GFDL-ESM2M,2036_2040
7,SiskiyouForest,GFDL-ESM2M,2041_2045
11,SiskiyouForest,GFDL-ESM2M,2046_2050
15,SiskiyouForest,GFDL-ESM2M,2051_2055
19,SiskiyouForest,GFDL-ESM2M,2056_2060
23,SiskiyouForest,GFDL-ESM2M,2061_2065


<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [13]:
# select the Siskiyou rows from the maca_pr_2080_df
siskyou_2080_df = maca_pr_2080_df[maca_pr_2080_df['site_name']=='SiskiyouForest']

# select the CanESM2 rows from the siskyou_2080_df
siskiyou_2080_can_df = siskyou_2080_df[siskyou_2080_df['climate_model']=='CanESM2']

# select the MIROC-ESM-CHEM rows from the siskyou_2080_df
siskiyou_2080_miroc_df = siskyou_2080_df[siskyou_2080_df['climate_model']=='MIROC-ESM-CHEM']

# select the MRI-CGCM3 rows from the siskyou_2080_df
siskiyou_2080_mri_df = siskyou_2080_df[siskyou_2080_df['climate_model']=='MRI-CGCM3']

# select the GFDL-ESM2M	 rows from the siskyou_2080_df
siskiyou_2080_gfdl_df = siskyou_2080_df[siskyou_2080_df['climate_model']=='GFDL-ESM2M']

# Display df w/o DataArray column
display(siskiyou_2080_can_df.drop('da', axis='columns'))
display(siskiyou_2080_miroc_df.drop('da', axis='columns'))
display(siskiyou_2080_mri_df.drop('da', axis='columns'))
display(siskiyou_2080_gfdl_df.drop('da', axis='columns'))

# Pull out the data array for the first row from the CanESM2 df
siskiyou_2080_can_df.da.values[0]

,site_name,climate_model,date_range
0,SiskiyouForest,CanESM2,2066_2070
4,SiskiyouForest,CanESM2,2071_2075
8,SiskiyouForest,CanESM2,2076_2080
12,SiskiyouForest,CanESM2,2081_2085
16,SiskiyouForest,CanESM2,2086_2090
20,SiskiyouForest,CanESM2,2091_2095


,site_name,climate_model,date_range
1,SiskiyouForest,MIROC-ESM-CHEM,2066_2070
5,SiskiyouForest,MIROC-ESM-CHEM,2071_2075
9,SiskiyouForest,MIROC-ESM-CHEM,2076_2080
13,SiskiyouForest,MIROC-ESM-CHEM,2081_2085
17,SiskiyouForest,MIROC-ESM-CHEM,2086_2090
21,SiskiyouForest,MIROC-ESM-CHEM,2091_2095


,site_name,climate_model,date_range
2,SiskiyouForest,MRI-CGCM3,2066_2070
6,SiskiyouForest,MRI-CGCM3,2071_2075
10,SiskiyouForest,MRI-CGCM3,2076_2080
14,SiskiyouForest,MRI-CGCM3,2081_2085
18,SiskiyouForest,MRI-CGCM3,2086_2090
22,SiskiyouForest,MRI-CGCM3,2091_2095


,site_name,climate_model,date_range
3,SiskiyouForest,GFDL-ESM2M,2066_2070
7,SiskiyouForest,GFDL-ESM2M,2071_2075
11,SiskiyouForest,GFDL-ESM2M,2076_2080
15,SiskiyouForest,GFDL-ESM2M,2081_2085
19,SiskiyouForest,GFDL-ESM2M,2086_2090
23,SiskiyouForest,GFDL-ESM2M,2091_2095


<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [14]:
# select the Los Padres rows from the maca_pr_2050_df
padres_2050_df = maca_pr_2050_df[maca_pr_2050_df['site_name']=='LosPadresForest']

# select the CanESM2 rows from the padres_2050_df
padres_2050_can_df = padres_2050_df[padres_2050_df['climate_model']=='CanESM2']

# select the MIROC-ESM-CHEM rows from the padres_2050_df
padres_2050_miroc_df = padres_2050_df[padres_2050_df['climate_model']=='MIROC-ESM-CHEM']

# select the MRI-CGCM3 rows from the padres_2050_df
padres_2050_mri_df = padres_2050_df[padres_2050_df['climate_model']=='MRI-CGCM3']

# select the GFDL-ESM2M	 rows from the padres_2050_df
padres_2050_gfdl_df = padres_2050_df[padres_2050_df['climate_model']=='GFDL-ESM2M']

# Display df w/o DataArray column
display(padres_2050_can_df.drop('da', axis='columns'))
display(padres_2050_miroc_df.drop('da', axis='columns'))
display(padres_2050_mri_df.drop('da', axis='columns'))
display(padres_2050_gfdl_df.drop('da', axis='columns'))

# Pull out the data array for the first row from the CanESM2 df
padres_2050_can_df.da.values[0]

,site_name,climate_model,date_range
24,LosPadresForest,CanESM2,2036_2040
28,LosPadresForest,CanESM2,2041_2045
32,LosPadresForest,CanESM2,2046_2050
36,LosPadresForest,CanESM2,2051_2055
40,LosPadresForest,CanESM2,2056_2060
44,LosPadresForest,CanESM2,2061_2065


,site_name,climate_model,date_range
25,LosPadresForest,MIROC-ESM-CHEM,2036_2040
29,LosPadresForest,MIROC-ESM-CHEM,2041_2045
33,LosPadresForest,MIROC-ESM-CHEM,2046_2050
37,LosPadresForest,MIROC-ESM-CHEM,2051_2055
41,LosPadresForest,MIROC-ESM-CHEM,2056_2060
45,LosPadresForest,MIROC-ESM-CHEM,2061_2065


,site_name,climate_model,date_range
26,LosPadresForest,MRI-CGCM3,2036_2040
30,LosPadresForest,MRI-CGCM3,2041_2045
34,LosPadresForest,MRI-CGCM3,2046_2050
38,LosPadresForest,MRI-CGCM3,2051_2055
42,LosPadresForest,MRI-CGCM3,2056_2060
46,LosPadresForest,MRI-CGCM3,2061_2065


,site_name,climate_model,date_range
27,LosPadresForest,GFDL-ESM2M,2036_2040
31,LosPadresForest,GFDL-ESM2M,2041_2045
35,LosPadresForest,GFDL-ESM2M,2046_2050
39,LosPadresForest,GFDL-ESM2M,2051_2055
43,LosPadresForest,GFDL-ESM2M,2056_2060
47,LosPadresForest,GFDL-ESM2M,2061_2065


<xarray.DataArray 'precipitation' (time: 60, lat: 49, lon: 76)> Size: 894kB
[223440 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 392B 34.4 34.44 34.48 34.52 ... 36.31 36.35 36.4
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 608B -121.9 -121.8 -121.8 ... -118.8 -118.8 -118.7
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [15]:
# select the Los Padres rows from the maca_pr_2080_df
padres_2080_df = maca_pr_2080_df[maca_pr_2080_df['site_name']=='LosPadresForest']

# select the CanESM2 rows from the padres_2080_df
padres_2080_can_df = padres_2080_df[padres_2080_df['climate_model']=='CanESM2']

# select the MIROC-ESM-CHEM rows from the padres_2080_df
padres_2080_miroc_df = padres_2080_df[padres_2080_df['climate_model']=='MIROC-ESM-CHEM']

# select the MRI-CGCM3 rows from the padres_2080_df
padres_2080_mri_df = padres_2080_df[padres_2080_df['climate_model']=='MRI-CGCM3']

# select the GFDL-ESM2M	 rows from the padres_2080_df
padres_2080_gfdl_df = padres_2080_df[padres_2080_df['climate_model']=='GFDL-ESM2M']

# Display df w/o DataArray column
display(padres_2080_can_df.drop('da', axis='columns'))
display(padres_2080_miroc_df.drop('da', axis='columns'))
display(padres_2080_mri_df.drop('da', axis='columns'))
display(padres_2080_gfdl_df.drop('da', axis='columns'))

# Pull out the data array for the first row from the CanESM2 df
padres_2080_can_df.da.values[0]

,site_name,climate_model,date_range
24,LosPadresForest,CanESM2,2066_2070
28,LosPadresForest,CanESM2,2071_2075
32,LosPadresForest,CanESM2,2076_2080
36,LosPadresForest,CanESM2,2081_2085
40,LosPadresForest,CanESM2,2086_2090
44,LosPadresForest,CanESM2,2091_2095


,site_name,climate_model,date_range
25,LosPadresForest,MIROC-ESM-CHEM,2066_2070
29,LosPadresForest,MIROC-ESM-CHEM,2071_2075
33,LosPadresForest,MIROC-ESM-CHEM,2076_2080
37,LosPadresForest,MIROC-ESM-CHEM,2081_2085
41,LosPadresForest,MIROC-ESM-CHEM,2086_2090
45,LosPadresForest,MIROC-ESM-CHEM,2091_2095


,site_name,climate_model,date_range
26,LosPadresForest,MRI-CGCM3,2066_2070
30,LosPadresForest,MRI-CGCM3,2071_2075
34,LosPadresForest,MRI-CGCM3,2076_2080
38,LosPadresForest,MRI-CGCM3,2081_2085
42,LosPadresForest,MRI-CGCM3,2086_2090
46,LosPadresForest,MRI-CGCM3,2091_2095


,site_name,climate_model,date_range
27,LosPadresForest,GFDL-ESM2M,2066_2070
31,LosPadresForest,GFDL-ESM2M,2071_2075
35,LosPadresForest,GFDL-ESM2M,2076_2080
39,LosPadresForest,GFDL-ESM2M,2081_2085
43,LosPadresForest,GFDL-ESM2M,2086_2090
47,LosPadresForest,GFDL-ESM2M,2091_2095


<xarray.DataArray 'precipitation' (time: 60, lat: 49, lon: 76)> Size: 894kB
[223440 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 392B 34.4 34.44 34.48 34.52 ... 36.31 36.35 36.4
  * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
  * lon      (lon) float64 608B -121.9 -121.8 -121.8 ... -118.8 -118.8 -118.7
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<span style="color:purple">

I want to know:

* For each site,
* for each time period,
* for each climate model,

what is the predicted average annual precipitation?

So for example, what is the predicted average annual precipitation for 2050 for Siskiyou according to the CanESM2 model?

What is the predicted average annual precipitation for 2050 for Siskiyou according to the MIROC-ESM-CHEM model?

What is the predicated average annual precipitation for 2050 for Los Padres according to the CanESM2 model?

What is the predicted average annual precipitation for 2050 for Los Padres according to the MIROC-ESM-CHEM model?

etc...


</span>

# **BREAK**

**I feel confident about the code above.**

What all did I try below?

### First, I put the 16 subsetted DataFrames into a dictionary so that I could theoretically loop through them.

In [16]:
# create dictionary of all the subsetted DataFrames
climate_df_dict = {
    'Siskiyou 2050 CanESM2' : siskiyou_2050_can_df,
    'Siskiyou 2050 MIROC-ESM-CHEM' : siskiyou_2050_miroc_df,
    'Siskiyou 2050 MRI-CGCM3' : siskiyou_2050_mri_df,
    'Siskiyou 2050 GFDL-ESM2M' : siskiyou_2050_gfdl_df,
    'Siskiyou 2080 CanESM2' : siskiyou_2080_can_df,
    'Siskiyou 2080 MIROC-ESM-CHEM' : siskiyou_2080_miroc_df,
    'Siskiyou 2080 MRI-CGCM3' : siskiyou_2080_mri_df,
    'Siskiyou 2080 GFDL-ESM2M' : siskiyou_2080_gfdl_df,
    'Padres 2050 CanESM2' : padres_2050_can_df,
    'Padres 2050 MIROC-ESM-CHEM' : padres_2050_miroc_df,
    'Padres 2050 MRI-CGCM3' : padres_2050_mri_df,
    'Padres 2050 GFDL-ESM2M' : padres_2050_gfdl_df,
    'Padres 2080 CanESM2' : padres_2080_can_df,
    'Padres 2080 MIROC-ESM-CHEM' : padres_2080_miroc_df,
    'Padres 2080 MRI-CGCM3' : padres_2080_mri_df,
    'Padres 2080 GFDL-ESM2M' : padres_2080_gfdl_df  
}

# check the keys of the dict were created correctly
climate_df_dict.keys()

dict_keys(['Siskiyou 2050 CanESM2', 'Siskiyou 2050 MIROC-ESM-CHEM', 'Siskiyou 2050 MRI-CGCM3', 'Siskiyou 2050 GFDL-ESM2M', 'Siskiyou 2080 CanESM2', 'Siskiyou 2080 MIROC-ESM-CHEM', 'Siskiyou 2080 MRI-CGCM3', 'Siskiyou 2080 GFDL-ESM2M', 'Padres 2050 CanESM2', 'Padres 2050 MIROC-ESM-CHEM', 'Padres 2050 MRI-CGCM3', 'Padres 2050 GFDL-ESM2M', 'Padres 2080 CanESM2', 'Padres 2080 MIROC-ESM-CHEM', 'Padres 2080 MRI-CGCM3', 'Padres 2080 GFDL-ESM2M'])

### Second, I spent time figuring out how to access the DataArrays in the DataFrames in the climate_df_dict.

In [17]:
# display the dataframe for the key 'Siskiyou 2080 CanESM2'
# without the 'da' column b/c that takes forever to display
display(climate_df_dict.get('Siskiyou 2080 CanESM2').drop('da', axis='columns'))

,site_name,climate_model,date_range
0,SiskiyouForest,CanESM2,2066_2070
4,SiskiyouForest,CanESM2,2071_2075
8,SiskiyouForest,CanESM2,2076_2080
12,SiskiyouForest,CanESM2,2081_2085
16,SiskiyouForest,CanESM2,2086_2090
20,SiskiyouForest,CanESM2,2091_2095


In [18]:
# this displays the each of the 16 dataframes w/o the 'da' column
# for i in climate_df_dict.keys():
#     display(climate_df_dict.get(i).drop('da', axis='columns'))

# Access the dataarray for the first row of each of the climate_df_dict dataframes
for i in climate_df_dict.keys():
    display(climate_df_dict.get(i).da.values[0])

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

...

<xarray.DataArray 'precipitation' (time: 60, lat: 49, lon: 76)> Size: 894kB
[223440 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 392B 34.4 34.44 34.48 34.52 ... 36.31 36.35 36.4
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 608B -121.9 -121.8 -121.8 ... -118.8 -118.8 -118.7
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<xarray.DataArray 'precipitation' (time: 60, lat: 49, lon: 76)> Size: 894kB
[223440 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 392B 34.4 34.44 34.48 34.52 ... 36.31 36.35 36.4
  * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
  * lon      (lon) float64 608B -121.9 -121.8 -121.8 ... -118.8 -118.8 -118.7
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<xarray.DataArray 'precipitation' (time: 60, lat: 49, lon: 76)> Size: 894kB
[223440 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 392B 34.4 34.44 34.48 34.52 ... 36.31 36.35 36.4
  * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
  * lon      (lon) float64 608B -121.9 -121.8 -121.8 ... -118.8 -118.8 -118.7
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<xarray.DataArray 'precipitation' (time: 60, lat: 49, lon: 76)> Size: 894kB
[223440 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 392B 34.4 34.44 34.48 34.52 ... 36.31 36.35 36.4
  * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
  * lon      (lon) float64 608B -121.9 -121.8 -121.8 ... -118.8 -118.8 -118.7
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<xarray.DataArray 'precipitation' (time: 60, lat: 49, lon: 76)> Size: 894kB
[223440 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 392B 34.4 34.44 34.48 34.52 ... 36.31 36.35 36.4
  * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
  * lon      (lon) float64 608B -121.9 -121.8 -121.8 ... -118.8 -118.8 -118.7
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [19]:
# Access the dataarray for the each row of each of the climate_df_dict dataframes
# should have 96 dataarrays
# it works! 
# with the for statements in this order, one loop through gives the
# DataArray for the first row of each of the climate_df_dict dataframes, so 16 dataarrays.
# If I loop through all 6 times, I get 96 dataarrays.
# What happens if I switch the order of the for statements?
for i in tqdm(climate_df_dict.keys()):
    for num in tqdm(range(6)):
        display(climate_df_dict.get(i).da.values[num])
        break

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

  0%|          | 0/6 [00:00<?, ?it/s]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

...

<xarray.DataArray 'precipitation' (time: 60, lat: 49, lon: 76)> Size: 894kB
[223440 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 392B 34.4 34.44 34.48 34.52 ... 36.31 36.35 36.4
  * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
  * lon      (lon) float64 608B -121.9 -121.8 -121.8 ... -118.8 -118.8 -118.7
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

  0%|          | 0/6 [00:00<?, ?it/s]

<xarray.DataArray 'precipitation' (time: 60, lat: 49, lon: 76)> Size: 894kB
[223440 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 392B 34.4 34.44 34.48 34.52 ... 36.31 36.35 36.4
  * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
  * lon      (lon) float64 608B -121.9 -121.8 -121.8 ... -118.8 -118.8 -118.7
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

  0%|          | 0/6 [00:00<?, ?it/s]

<xarray.DataArray 'precipitation' (time: 60, lat: 49, lon: 76)> Size: 894kB
[223440 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 392B 34.4 34.44 34.48 34.52 ... 36.31 36.35 36.4
  * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
  * lon      (lon) float64 608B -121.9 -121.8 -121.8 ... -118.8 -118.8 -118.7
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [20]:
# create a list of the climate_df_dict.keys()
dict_keys = list(climate_df_dict.keys())
dict_keys

['Siskiyou 2050 CanESM2',
 'Siskiyou 2050 MIROC-ESM-CHEM',
 'Siskiyou 2050 MRI-CGCM3',
 'Siskiyou 2050 GFDL-ESM2M',
 'Siskiyou 2080 CanESM2',
 'Siskiyou 2080 MIROC-ESM-CHEM',
 'Siskiyou 2080 MRI-CGCM3',
 'Siskiyou 2080 GFDL-ESM2M',
 'Padres 2050 CanESM2',
 'Padres 2050 MIROC-ESM-CHEM',
 'Padres 2050 MRI-CGCM3',
 'Padres 2050 GFDL-ESM2M',
 'Padres 2080 CanESM2',
 'Padres 2080 MIROC-ESM-CHEM',
 'Padres 2080 MRI-CGCM3',
 'Padres 2080 GFDL-ESM2M']

### Third, once I figured out how to access the DataArrays in the DataFrames in the climate_df_dict w/ the for statements in the correct order, I worked on calculating the average annual precipitation amounts for each 5 year period, AKA the average annual precipitation over each row of the 6 row DataFrame. With the break in the loop below in row 28, I got 6 DataArrays in a list, presumably all for the 'Siskiyou2050 CanESM2' key of the climate_df_dict since that's the first key. 

#### I am unsure of how I could take the break out of the loop below and be able to group each set of 6 DataArrays together so that I could then calculate the average annual precipitation for the whole 30 year period for one DataFrame. How could I tell the loop that the first 6 DataArrays are for 'Siskiyou 2050 CanESM2', the second 6 are for 'Siskiyou 2050 MIROC-ESM-CHEM', the third set of 6 are for 'Siskiyou 2050 MRI-CGCM3', etc so that I could then take the mean of each set of 6? 

##### Do I need to move where the ave_yearly_precip_das = [] is so that that list holds each set of 6? So I'd have 16 ave_yearly_precip_das lists? But then how could I label each list? Maybe I need to use a dictionary? not sure how to make that work.

In [21]:
# Switching the order of the for statements to
# Access the dataarray for the each row of each of the climate_df_dict dataframes
# should have 96 dataarrays
# With the order switched, I now get 6 dataarrays in a list when I put a break in on line 26,
# presumably all for the 'Siskiyou 2050 CanESM2' key.
# Each of the 6 DataArrays represents the ave annual precip of a 5 year period of the Siskisyou 2050 CanESM2 DataFrame
# I think this is what I need b/c now I could use these 6 dataarrays to find the average annual precipitation for the 2036-2065 time period

# empty list to store ave_yearly_precip_das
ave_yearly_precip_das = []
for num in tqdm(range(6)):
    for i in tqdm(climate_df_dict.keys()):

        # access the DataArray for each row of the climate_df_dict DataFrames
        da = climate_df_dict.get(i).da.values[num]

        # add a CRS and set the spatial dimensions of the da
        crs_spatial_da = da.rio.write_crs(4326).rio.set_spatial_dims('lat', 'lon')

        # calculate the total precip for each year of the five year period of the crs_spatial_da
        yearly_precip_da = crs_spatial_da.groupby('time.year').sum()

        # calculate the average annual precip for each year of the 5 year time period
        ave_yearly_precip_da = yearly_precip_da.mean(['year'])

        # add ave_yearly_precip_da to ave_yearly_precip_das list
        ave_yearly_precip_das.append(ave_yearly_precip_da)
        break
ave_yearly_precip_das

### HOW COULD I PUT EACH OF THE 6 DATARRAYS FOR THE 16 KEYS BACK INTO A DICTIONARY SO THAT I CAN FIND THE AVERAGE OVER THE WHOLE TIME PERIOD?
### ALTERNATIVELY, USE THE ABOVE CODE ON EACH OF THE SUBSETTED DATAFRAMES ABOVE TO FIND THE AVE ANNUAL PRECIP FOR EACH 5 YEAR DA
### IN EACH ROW OF THE SUBSETTED DATAFRAMES AND THEN IT MAY BE EASIER TO FIND THE AVE ANNUAL PRECIP FOR EACH 30 YEAR PERIOD

### WHAT IF I ADDED SOMETHING LIKE LINE 22 TO FIND THE 30 YEAR AVERAGE BUT INSTEAD OF MEAN.(['YEAR']) IT WAS MEAN.(['MODEL'])?



  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

[<xarray.DataArray 'precipitation' (lat: 25, lon: 27)> Size: 3kB
 array([[   0.     ,    0.     ,    0.     ,    0.     , 1939.9421 ,
         2008.6213 , 2117.7676 , 2574.5688 , 3053.769  , 3023.8066 ,
         3051.2583 , 3453.1646 , 3368.2417 , 2379.092  , 2251.679  ,
         2239.9656 , 2510.95   , 2853.609  , 2970.7642 , 2867.7952 ,
         2405.228  , 2183.7139 , 1864.6106 , 1776.5625 , 2116.8162 ,
         1958.3151 , 1924.9973 ],
        [   0.     ,    0.     ,    0.     ,    0.     , 1946.9934 ,
         2164.559  , 2361.3984 , 2509.7656 , 2762.7878 , 3018.4026 ,
         3135.6433 , 3127.1973 , 3061.6328 , 2600.2104 , 2395.1797 ,
         2354.3027 , 2349.307  , 2574.868  , 2554.7996 , 2317.0703 ,
         2390.6304 , 2227.9585 , 1915.0424 , 1947.6217 , 2058.817  ,
         1775.8041 , 1826.7797 ],
        [   0.     ,    0.     , 1938.6588 , 1900.4353 , 1943.1031 ,
         2307.5955 , 2551.1157 , 2694.5737 , 2772.0176 , 3227.3853 ,
         3045.9517 , 3109.9792 , 3058.3

### So, because I can't currently figure out how to get my loop above to work for all 16 subsetted DataFrames at once, I'll define the loop as a function and use it on each of the subsetted DataFrames individually.

### Before I do that, I used chatgpt to come up with line 6 of the code below. Line 6 is using the ave_yearly_precip_das list created above, which contains the 6 5-year predicted precip average DataArrays for the 'Siskiyou 2050 CanESM2' DataFrame, to then find the average of those 6 DataArrays, AKA the predicted annual average precip over the 30 year period of 2035 to 2065, AKA 2050.

In [22]:
# calculate the average of all 6 dataarrays for siskiyou 2050 canEsm2
# xr.concat() joins DataArrays in ave_yearly_precip_das along the new dimension 'stack'
# .mean(dim='stack') calculates the ave of al DataArrays along the 'stack' dimension
# now, instead of 6 DataArrays for the Siskiyou CanESM2 2050 DataFrame, each representing the 5 year annual precip ave for each row of the dataframe,
# I have 1 DataArray representing the 30 year annual precipitation ave for the whole Siskiyou CanESM2 2050 DataFrame
ave_annual_precip_sis_can_2050_da = xr.concat(ave_yearly_precip_das, dim='stack').mean(dim='stack')
ave_annual_precip_sis_can_2050_da

<xarray.DataArray 'precipitation' (lat: 25, lon: 27)> Size: 3kB
array([[   0.     ,    0.     ,    0.     ,    0.     , 1952.9447 ,
        2032.3126 , 2147.47   , 2637.7673 , 3140.83   , 3105.4138 ,
        3137.6006 , 3578.9834 , 3485.7888 , 2448.2295 , 2312.9841 ,
        2296.0469 , 2582.7412 , 2979.5996 , 3096.3906 , 2990.4475 ,
        2502.5588 , 2259.0793 , 1932.8556 , 1832.287  , 2168.911  ,
        2001.5524 , 1964.2653 ],
       [   0.     ,    0.     ,    0.     ,    0.     , 1965.854  ,
        2196.369  , 2411.179  , 2575.1716 , 2834.4023 , 3101.044  ,
        3235.4187 , 3231.2717 , 3163.784  , 2676.8977 , 2458.048  ,
        2408.6758 , 2399.6636 , 2642.8174 , 2651.429  , 2401.986  ,
        2475.108  , 2301.0457 , 1983.476  , 2039.6647 , 2129.43   ,
        1814.4025 , 1866.5381 ],
       [   0.     ,    0.     , 1949.1058 , 1923.6237 , 1971.9346 ,
        2357.5762 , 2616.6736 , 2771.6248 , 2848.424  , 3334.1028 ,
        3139.0918 , 3216.8691 , 3160.738  , 3011.9646 , 2765.947  ,
        2627.1768 , 2333.5264 , 2469.6133 , 2196.3628 , 1870.7777 ,
        2331.2249 , 2273.1956 , 2199.07   , 2165.2017 , 1986.9388 ,
        1807.7119 , 1803.9784 ],
       [   0.     , 1902.7964 , 2023.0663 , 2015.267  , 2330.9822 ,
        2414.3872 , 2583.9326 , 2595.3865 , 2803.1448 , 3312.6028 ,
...
        1140.8472 , 1159.8232 , 1210.9742 , 1226.2501 , 1181.997  ,
        1114.009  , 1048.1633 ],
       [2613.2427 , 3006.1885 , 3066.299  , 3009.1572 , 2502.4695 ,
        2339.241  , 2380.6064 , 2349.83   , 2027.8221 , 1998.1075 ,
        2183.46   , 2277.226  , 2401.985  , 2003.3043 , 1665.8383 ,
        1377.4854 , 1296.8116 , 1256.598  , 1214.5465 , 1185.2555 ,
        1167.5474 , 1153.3317 , 1114.4347 , 1166.4493 , 1190.1956 ,
        1140.4257 , 1115.57   ],
       [3085.0332 , 3273.084  , 3261.8835 , 2766.3887 , 2124.03   ,
        2049.4165 , 2153.848  , 1984.8433 , 1641.2853 , 1868.666  ,
        1981.8744 , 2166.638  , 2496.3857 , 2214.6235 , 1477.1035 ,
        1287.2314 , 1269.6722 , 1288.2115 , 1252.0375 , 1188.6046 ,
        1151.8363 , 1149.2023 , 1112.9171 , 1045.7777 , 1097.0231 ,
        1052.2758 ,  985.1125 ],
       [2563.2915 , 2709.7842 , 2606.1355 , 2442.751  , 2053.2168 ,
        1895.9736 , 1794.9746 , 1703.5934 , 1688.0092 , 1810.9896 ,
        1835.42   , 2036.2529 , 2155.5032 , 2093.2488 , 1539.7875 ,
        1276.7535 , 1261.7261 , 1274.5109 , 1249.4708 , 1207.9242 ,
        1138.6526 , 1109.1627 , 1049.6041 ,  992.4954 ,  954.51276,
         950.5779 ,  972.03076]], dtype=float32)
Coordinates:
  * lat          (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * lon          (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.3
    crs          int64 8B 0
    spatial_ref  int64 8B 0

### Start by focusing on the Siskiyou 2050 climate models:

In [23]:
# Display df w/o DataArray column
display(siskiyou_2050_can_df.drop('da', axis='columns'))
display(siskiyou_2050_miroc_df.drop('da', axis='columns'))
display(siskiyou_2050_mri_df.drop('da', axis='columns'))
display(siskiyou_2050_gfdl_df.drop('da', axis='columns'))

# Pull out the data array for the first row from the CanESM2 df
siskiyou_2050_can_df.da.values[0]

,site_name,climate_model,date_range
0,SiskiyouForest,CanESM2,2036_2040
4,SiskiyouForest,CanESM2,2041_2045
8,SiskiyouForest,CanESM2,2046_2050
12,SiskiyouForest,CanESM2,2051_2055
16,SiskiyouForest,CanESM2,2056_2060
20,SiskiyouForest,CanESM2,2061_2065


,site_name,climate_model,date_range
1,SiskiyouForest,MIROC-ESM-CHEM,2036_2040
5,SiskiyouForest,MIROC-ESM-CHEM,2041_2045
9,SiskiyouForest,MIROC-ESM-CHEM,2046_2050
13,SiskiyouForest,MIROC-ESM-CHEM,2051_2055
17,SiskiyouForest,MIROC-ESM-CHEM,2056_2060
21,SiskiyouForest,MIROC-ESM-CHEM,2061_2065


,site_name,climate_model,date_range
2,SiskiyouForest,MRI-CGCM3,2036_2040
6,SiskiyouForest,MRI-CGCM3,2041_2045
10,SiskiyouForest,MRI-CGCM3,2046_2050
14,SiskiyouForest,MRI-CGCM3,2051_2055
18,SiskiyouForest,MRI-CGCM3,2056_2060
22,SiskiyouForest,MRI-CGCM3,2061_2065


,site_name,climate_model,date_range
3,SiskiyouForest,GFDL-ESM2M,2036_2040
7,SiskiyouForest,GFDL-ESM2M,2041_2045
11,SiskiyouForest,GFDL-ESM2M,2046_2050
15,SiskiyouForest,GFDL-ESM2M,2051_2055
19,SiskiyouForest,GFDL-ESM2M,2056_2060
23,SiskiyouForest,GFDL-ESM2M,2061_2065


<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [24]:
siskiyou_2050_can_df.da.values[3]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2051-01-15 00:00:00 ... 2055-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [25]:
# Define a new loop to calculate the ave annaul precip for each 5 year period in each row of a subsetted DataFrame

# empty list to store ave_yearly_precip_das
ave_annual_precip_5yrs_das = []

# for each of the 6 rows in the DataFrame,
for num in tqdm(range(6)):
    #access the DataArray in that row
    row_da = siskiyou_2050_can_df.da.values[num]

    # add a CRS and set the spatial dimensions of the row_da
    crs_spatial_da = row_da.rio.write_crs(4326).rio.set_spatial_dims('lat', 'lon')

    # calculate the total precip for each year of the five year period of the crs_spatial_da
    yearly_precip_da = crs_spatial_da.groupby('time.year').sum()

    # calculate the average annual precip for each year of the 5 year time period
    ave_yearly_precip_da = yearly_precip_da.mean(['year'])

    # add ave_yearly_precip_da to ave_annual_precip_5yrs_das list
    ave_annual_precip_5yrs_das.append(ave_yearly_precip_da)

ave_annual_precip_5yrs_das

  0%|          | 0/6 [00:00<?, ?it/s]

[<xarray.DataArray 'precipitation' (lat: 25, lon: 27)> Size: 3kB
 array([[   0.     ,    0.     ,    0.     ,    0.     , 1939.9421 ,
         2008.6213 , 2117.7676 , 2574.5688 , 3053.769  , 3023.8066 ,
         3051.2583 , 3453.1646 , 3368.2417 , 2379.092  , 2251.679  ,
         2239.9656 , 2510.95   , 2853.609  , 2970.7642 , 2867.7952 ,
         2405.228  , 2183.7139 , 1864.6106 , 1776.5625 , 2116.8162 ,
         1958.3151 , 1924.9973 ],
        [   0.     ,    0.     ,    0.     ,    0.     , 1946.9934 ,
         2164.559  , 2361.3984 , 2509.7656 , 2762.7878 , 3018.4026 ,
         3135.6433 , 3127.1973 , 3061.6328 , 2600.2104 , 2395.1797 ,
         2354.3027 , 2349.307  , 2574.868  , 2554.7996 , 2317.0703 ,
         2390.6304 , 2227.9585 , 1915.0424 , 1947.6217 , 2058.817  ,
         1775.8041 , 1826.7797 ],
        [   0.     ,    0.     , 1938.6588 , 1900.4353 , 1943.1031 ,
         2307.5955 , 2551.1157 , 2694.5737 , 2772.0176 , 3227.3853 ,
         3045.9517 , 3109.9792 , 3058.3

In [26]:
# using the new loop above,
# calculate the average of all 6 dataarrays for siskiyou 2050 canEsm2
# xr.concat() joins DataArrays in ave_annual_precip_5yrs_das along the new dimension 'stack'
# .mean(dim='stack') calculates the ave of al DataArrays along the 'stack' dimension
# now, instead of 6 DataArrays for the Siskiyou CanESM2 2050 DataFrame, each representing the 5 year annual precip ave for each row of the dataframe,
# I have 1 DataArray representing the 30 year annual precipitation ave for the whole Siskiyou CanESM2 2050 DataFrame
ave_annual_precip_sis_can_2050_da = xr.concat(ave_annual_precip_5yrs_das, dim='stack').mean(dim='stack')
ave_annual_precip_sis_can_2050_da

<xarray.DataArray 'precipitation' (lat: 25, lon: 27)> Size: 3kB
array([[   0.     ,    0.     ,    0.     ,    0.     , 1952.9447 ,
        2032.3126 , 2147.47   , 2637.7673 , 3140.83   , 3105.4138 ,
        3137.6006 , 3578.9834 , 3485.7888 , 2448.2295 , 2312.9841 ,
        2296.0469 , 2582.7412 , 2979.5996 , 3096.3906 , 2990.4475 ,
        2502.5588 , 2259.0793 , 1932.8556 , 1832.287  , 2168.911  ,
        2001.5524 , 1964.2653 ],
       [   0.     ,    0.     ,    0.     ,    0.     , 1965.854  ,
        2196.369  , 2411.179  , 2575.1716 , 2834.4023 , 3101.044  ,
        3235.4187 , 3231.2717 , 3163.784  , 2676.8977 , 2458.048  ,
        2408.6758 , 2399.6636 , 2642.8174 , 2651.429  , 2401.986  ,
        2475.108  , 2301.0457 , 1983.476  , 2039.6647 , 2129.43   ,
        1814.4025 , 1866.5381 ],
       [   0.     ,    0.     , 1949.1058 , 1923.6237 , 1971.9346 ,
        2357.5762 , 2616.6736 , 2771.6248 , 2848.424  , 3334.1028 ,
        3139.0918 , 3216.8691 , 3160.738  , 3011.9646 , 2765.947  ,
        2627.1768 , 2333.5264 , 2469.6133 , 2196.3628 , 1870.7777 ,
        2331.2249 , 2273.1956 , 2199.07   , 2165.2017 , 1986.9388 ,
        1807.7119 , 1803.9784 ],
       [   0.     , 1902.7964 , 2023.0663 , 2015.267  , 2330.9822 ,
        2414.3872 , 2583.9326 , 2595.3865 , 2803.1448 , 3312.6028 ,
...
        1140.8472 , 1159.8232 , 1210.9742 , 1226.2501 , 1181.997  ,
        1114.009  , 1048.1633 ],
       [2613.2427 , 3006.1885 , 3066.299  , 3009.1572 , 2502.4695 ,
        2339.241  , 2380.6064 , 2349.83   , 2027.8221 , 1998.1075 ,
        2183.46   , 2277.226  , 2401.985  , 2003.3043 , 1665.8383 ,
        1377.4854 , 1296.8116 , 1256.598  , 1214.5465 , 1185.2555 ,
        1167.5474 , 1153.3317 , 1114.4347 , 1166.4493 , 1190.1956 ,
        1140.4257 , 1115.57   ],
       [3085.0332 , 3273.084  , 3261.8835 , 2766.3887 , 2124.03   ,
        2049.4165 , 2153.848  , 1984.8433 , 1641.2853 , 1868.666  ,
        1981.8744 , 2166.638  , 2496.3857 , 2214.6235 , 1477.1035 ,
        1287.2314 , 1269.6722 , 1288.2115 , 1252.0375 , 1188.6046 ,
        1151.8363 , 1149.2023 , 1112.9171 , 1045.7777 , 1097.0231 ,
        1052.2758 ,  985.1125 ],
       [2563.2915 , 2709.7842 , 2606.1355 , 2442.751  , 2053.2168 ,
        1895.9736 , 1794.9746 , 1703.5934 , 1688.0092 , 1810.9896 ,
        1835.42   , 2036.2529 , 2155.5032 , 2093.2488 , 1539.7875 ,
        1276.7535 , 1261.7261 , 1274.5109 , 1249.4708 , 1207.9242 ,
        1138.6526 , 1109.1627 , 1049.6041 ,  992.4954 ,  954.51276,
         950.5779 ,  972.03076]], dtype=float32)
Coordinates:
  * lat          (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * lon          (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.3
    crs          int64 8B 0
    spatial_ref  int64 8B 0

### For the two previous code cells, do I have to loop through them individually w/ each subsetted DataFrame? I am running into the same issue where I don't know how to use those loops to loop through each subsetted code cell and put the result somewhere that is associated with the name/characteristics of the subsetted DataFrame it came from. I feel like if I'm not able to do that, I'm going to have to run those two loops 16 times which is too much.

### try creating a list to loop through instead of a dictionary

In [31]:
climate_df_list = [
    siskiyou_2050_can_df,
    siskiyou_2050_miroc_df,
    siskiyou_2050_mri_df,
    siskiyou_2050_gfdl_df,
    siskiyou_2080_can_df,
    siskiyou_2080_miroc_df,
    siskiyou_2080_mri_df,
    siskiyou_2080_gfdl_df,
    padres_2050_can_df,
    padres_2050_miroc_df,
    padres_2050_mri_df,
    padres_2050_gfdl_df,
    padres_2080_can_df,
    padres_2080_miroc_df,
    padres_2080_mri_df,
    padres_2080_gfdl_df
]

In [ ]:
siskiyou_2050_can_df.da.values[0]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [30]:
len(siskiyou_2050_can_df.da.values[0])

60

In [ ]:
# list for results
yearly_averages = []

for year_index in range(len(climate_df_list.da.values[0])):
    # Extract precip for a specific year across all arrays
    year_precip = [climate_df_list[i][year_index] for i in range(16)]
    # Calculate the average for that year
    yearly_average = sum(year_precip) / len(year_precip)
    yearly_averages.append(yearly_average)

yearly_averages

# **TESTING CODE:**

In [ ]:
siskiyou_2050_climate_df_dict = {
    'Siskiyou 2050 CanESM2' : siskiyou_2050_can_df,
    'Siskiyou 2050 MIROC-ESM-CHEM' : siskiyou_2050_miroc_df,
    'Siskiyou 2050 MRI-CGCM3' : siskiyou_2050_mri_df,
    'Siskiyou 2050 GFDL-ESM2M' : siskiyou_2050_gfdl_df
}

siskiyou_2080_climate_df_dict = {
    'Siskiyou 2080 CanESM2' : siskiyou_2080_can_df,
    'Siskiyou 2080 MIROC-ESM-CHEM' : siskiyou_2080_miroc_df,
    'Siskiyou 2080 MRI-CGCM3' : siskiyou_2080_mri_df,
    'Siskiyou 2080 GFDL-ESM2M' : siskiyou_2080_gfdl_df
}

# check the keys of the dict were created correctly
siskiyou_2080_climate_df_dict.keys()

dict_keys(['Siskiyou 2080 CanESM2', 'Siskiyou 2080 MIROC-ESM-CHEM', 'Siskiyou 2080 MRI-CGCM3', 'Siskiyou 2080 GFDL-ESM2M'])

In [22]:
padres_2050_climate_df_dict = {
    'Padres 2050 CanESM2' : padres_2050_can_df,
    'Padres 2050 MIROC-ESM-CHEM' : padres_2050_miroc_df,
    'Padres 2050 MRI-CGCM3' : padres_2050_mri_df,
    'Padres 2050 GFDL-ESM2M' : padres_2050_gfdl_df
}

padres_2080_climate_df_dict = {
    'Padres 2080 CanESM2' : padres_2080_can_df,
    'Padres 2080 MIROC-ESM-CHEM' : padres_2080_miroc_df,
    'Padres 2080 MRI-CGCM3' : padres_2080_mri_df,
    'Padres 2080 GFDL-ESM2M' : padres_2080_gfdl_df
}

# check the keys of the dict were created correctly
padres_2080_climate_df_dict.keys()

dict_keys(['Padres 2080 CanESM2', 'Padres 2080 MIROC-ESM-CHEM', 'Padres 2080 MRI-CGCM3', 'Padres 2080 GFDL-ESM2M'])

In [ ]:
# Access the DataArray of the 2036_2040 date range of
# the 'Siskiyou 2050 CanESM2' key in the dict
siskiyou_2050_climate_df_dict['Siskiyou 2050 CanESM2'].da.values[0]

# Access the DataArray of the 2041_2045 date range of
# the 'Siskiyou 2050 CanESM2' key in the dict
siskiyou_2050_climate_df_dict['Siskiyou 2050 CanESM2'].da.values[1]

# Access the DataArray of the 2046_2050 date range of
# the 'Siskiyou 2050 CanESM2' key in the dict
siskiyou_2050_climate_df_dict['Siskiyou 2050 CanESM2'].da.values[2]

# Access the DataArray of the 2051_2055 date range of
# the 'Siskiyou 2050 CanESM2' key in the dict
siskiyou_2050_climate_df_dict['Siskiyou 2050 CanESM2'].da.values[3]

# Access the DataArray of the 2056_2060 date range of
# the 'Siskiyou 2050 CanESM2' key in the dict
siskiyou_2050_climate_df_dict['Siskiyou 2050 CanESM2'].da.values[4]

# Access the DataArray of the 2061_2065 date range of
# the 'Siskiyou 2050 CanESM2' key in the dict
siskiyou_2050_climate_df_dict['Siskiyou 2050 CanESM2'].da.values[5]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2061-01-15 00:00:00 ... 2065-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [21]:
for i in range(6):
               display(siskiyou_2080_climate_df_dict['Siskiyou 2080 CanESM2'].da.values[i])

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2071-01-15 00:00:00 ... 2075-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2076-01-15 00:00:00 ... 2080-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2081-01-15 00:00:00 ... 2085-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2086-01-15 00:00:00 ... 2090-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2091-01-15 00:00:00 ... 2095-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [ ]:
test_dict = {
    'test key 1':1,
    'test key 2': 2,
    'test key 3':3
}

In [ ]:
def subset_maca_df(site_name_long,
                   site_name_abbr
                   ):
    """
    Subset the maca_df by site, climate model, and year
    
    Parameters
    ----------
    site_name_long : str
        The name of the site as it appers in the site_name column of the maca_df
    site_name_abbr : str
        The abbreviation/shortened name of the site
    Returns
    -------
    
    """
    # select the site rows from the maca_pr_2050_df
    site_name_abbr_2050_df = maca_pr_2050_df[maca_pr_2050_df['site_name']=='site_name_long']

    # select the CanESM2 rows from the site_name_abbr_2050_df
    site_name_abbr_2050_can_df = site_name_abbr_2050_df[site_name_abbr_2050_df['climate_model']=='CanESM2']

    # select the MIROC-ESM-CHEM rows from the site_name_abbr_2050_df
    site_name_abbr_2050_miroc_df = site_name_abbr_2050_df[site_name_abbr_2050_df['climate_model']=='MIROC-ESM-CHEM']

    # select the MRI-CGCM3 rows from the site_name_abbr_2050_df
    site_name_abbr_2050_mri_df = site_name_abbr_2050_df[site_name_abbr_2050_df['climate_model']=='MRI-CGCM3']

    # select the GFDL-ESM2M	 rows from the site_name_abbr_2050_df
    site_name_abbr_2050_gfdl_df = site_name_abbr_2050_df[site_name_abbr_2050_df['climate_model']=='GFDL-ESM2M']

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Make sure to include a description of the climate data and how you
selected your models. Include a citation of the MACAv2 data</p></div></div>

YOUR CLIMATE DATA DESCRIPTION AND CITATIONS HERE